In [1]:
#####Multi-Way Rerational Graph Embedding model#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import math
from scipy import sparse
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize
from scipy.stats import norm
import seaborn as sns

#np.random.seed(98537)

In [2]:
####データの発生####
##データの設定
k = 10
d = 5000
m = 12
no = np.arange(d)

In [3]:
##ノードを生成
#セグメントをリンク確率を生成
g = 15
Z = np.random.multinomial(1, np.random.dirichlet(np.repeat(2.0, g), 1).reshape(-1), d)
z_vec = np.dot(Z, np.arange(g))
beta = np.random.beta(0.75, 15.0, g*g).reshape(g, g)
beta = np.triu(beta, k=1) + np.triu(beta).T

#ノードの全パターンを作成
target = np.triu(np.full((d, d), 1), k=1)
D1 = np.triu(np.full((d, d), np.arange(d)).T, k=1)
D2 = np.triu(np.full((d, d), np.arange(d)), k=1)
d1 = D1[target==1]; d2 = D2[target==1]
del target; D1, D2

#ユーザーごとにノードを生成
y = np.random.binomial(1, np.sum(beta[z_vec[d1], ] * Z[d2, ], axis=1), d1.shape[0])
index_y = np.where(y==1)[0]
d_id1 = d1[index_y]; d_id2 = d2[index_y]

#インデックスを作成
target = [i for i in range(d)]
d_list = [i for i in range(d)]
d_list1 = [i for i in range(d)]
d_list2 = [i for i in range(d)]
pt = np.repeat(0, d)
for i in range(d):
    d_list1[i] = np.array(np.where(d_id1==i)[0], dtype="int32")
    d_list2[i] = np.array(np.where(d_id2==i)[0], dtype="int32")
    d_list[i] = np.append(d_list1[i], d_list2[i])
    target[i] = np.append(np.repeat(0, d_list1[i].shape[0]), np.repeat(1, d_list2[i].shape[0]))
    pt[i] = target[i].shape[0]
N = np.sum(pt)
hhpt = d_id1.shape[0]

In [4]:
##説明変数の生成
#素性ベクトルを生成
k1 = 4; k2 = 5; k3 = 4
x1 = np.array(np.random.random(hhpt*k1)).reshape((hhpt, k1))
x2 = np.zeros((hhpt, k2))
for j in range(k2):
    prob = np.random.uniform(0.25, 0.55, 1)
    x2[:, j] = np.random.binomial(1, prob, hhpt)
x3 = np.random.multinomial(1, np.random.dirichlet(np.repeat(3.0, k3), 1).reshape(k3), hhpt)
x3 = np.delete(x3, np.argmin(np.sum(x3, axis=0)), axis=1)   #冗長な変数の削除
x = np.concatenate((np.repeat(1, hhpt)[:, np.newaxis], x1, x2, x3), axis=1)
x_col = x.shape[1]

#階層モデルの説明変数を生成
k1 = 4; k2 = 5; k3 = 5
u1 = np.array(np.random.random(d*k1)).reshape((d, k1))
u2 = np.zeros((d, k2))
for j in range(k2):
    prob = np.random.uniform(0.25, 0.55, 1)
    u2[:, j] = np.random.binomial(1, prob, d)
u3 = np.random.multinomial(1, np.random.dirichlet(np.repeat(3.0, k3), 1).reshape(k3), d)
u3 = np.delete(u3, np.argmin(np.sum(u3, axis=0)), axis=1)   #冗長な変数の削除
u = np.concatenate((np.repeat(1, d)[:, np.newaxis], u1, u2, u3), axis=1)
u_col = u.shape[1]

In [5]:
####応答変数を生成####
rp = 0
while True:
    rp = rp + 1
    print(rp)
    
    ##パラメータを生成
    #階層モデルのパラメータを生成
    alpha = np.random.normal(0, 0.3, k*u_col).reshape(u_col, k)
    Cov = np.diag(np.repeat(0.1, k))
    alphat = alpha.copy(); Covt = Cov.copy()

    #モデルパラメータを生成
    beta0 = np.random.uniform(-2.0, -1.0, m)
    beta1 = np.random.normal(0, 0.3, (x_col-1)*m)
    beta = np.append(beta0, beta1).reshape(x_col, m)
    theta = np.dot(u, alpha) + np.random.multivariate_normal(np.repeat(0, k), Cov, d)
    omega = np.zeros((k, k, m))
    for j in range(m):
        omega[:, :, j] = np.random.normal(0, 0.25, k*k).reshape(k, k)
    betat = beta.copy(); thetat = theta.copy(); omegat = omega.copy()
        
    ##応答変数を生成
    #期待値を設定
    beta_mu = np.dot(x, beta)
    mu = np.zeros((hhpt, m))
    for j in range(m):
        mu[:, j] = beta_mu[:, j] + np.dot(np.dot(theta, omega[:, :, j])[d_id1, ] * theta[d_id2, ], np.repeat(1, k))

    #正規分布からバイナリー変数を生成
    U = np.random.normal(mu, 1)
    y = np.array(U > 0, dtype="int")
    UT = U.copy()
    
    #break条件
    if (np.min(np.mean(y, axis=0)) > 0.1) & (np.max(np.mean(y, axis=0)) < 0.5):
        break

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34


In [6]:
#生成したデータを確認
print(np.mean(y, axis=0))
print(np.sum(y, axis=0))

[0.11793791 0.36248531 0.164327   0.17192043 0.31538261 0.32807515
 0.1402189  0.1122931  0.12141096 0.16289978 0.32561397 0.13868563]
[ 53382 164071  74379  77816 142751 148496  63467  50827  54954  73733
 147382  62773]


In [7]:
####マルコフ連鎖モンテカルロ法でRESCALを推定####
##パラメータ推定のための関数
#切断正規分布の乱数を発生させる関数
def rtnorm(mu, sigma, a, b, n, k):
    FA = norm.cdf(a, mu, sigma)
    FB = norm.cdf(b, mu, sigma)
    if k==1:
        res = norm.ppf(np.random.uniform(0, 1, n)*(FB-FA)+FA, mu, sigma)
    else:
        res = norm.ppf(np.random.uniform(0, 1, n*k).reshape(n, k)*(FB-FA)+FA, mu, sigma)
    return res

#対数尤度関数の和を計算する関数
def loglike(omega, theta, theta_vec1, theta_vec2, Sigma, er, d_id1, d_id2, hhpt, k, m):
    #期待値を設定
    ARA = np.zeros((hhpt, m))
    for j in range(m):
        ARA[:, j] = np.dot(np.dot(theta, omega[:, :, j])[d_id1, ] * theta_vec2, np.repeat(1, k))    
    mu = beta_mu + ARA

    #対数尤度の和
    LLi = scipy.stats.norm.logpdf(er, mu, Sigma)
    LL = np.sum(LLi, axis=0)
    return LL

#ベイジアン多変量回帰モデルをギブスサンプリングする関数
def rmultireg(Y, X, inv_XXV, XY, Cov, ADelta, Deltabar, V, nu, n, col, k):
    #事後分布のパラメータを設定
    beta_mu = np.dot(inv_XXV, XY + np.dot(ADelta, Deltabar)).T.reshape(-1)   #平均ベクトル
    sigma = np.kron(Cov, inv_XXV)   #分散共分散行列

    #パラメータをサンプリング
    beta_vec = np.random.multivariate_normal(beta_mu, sigma, 1)
    beta = beta_vec.reshape(col, k, order='F')   #回帰行列に変換

    
    ##逆ウィシャート分布から分散共分散行列をサンプリング
    #モデル誤差を設定
    mu = np.dot(X, beta)
    er = Y - mu

    #逆ウィシャート分布のパラメータ
    IW_R = np.dot(er.T, er) + V
    Sn = n + nu

    #パラメータをサンプリング
    Cov = scipy.stats.invwishart.rvs(Sn, IW_R, 1)
    return beta, Cov

In [8]:
##アルゴリズムの設定
LL1 = -100000000   #対数尤度の初期値
R = 2000
keep = 2  
iter = 0
burnin = 250/keep
disp = 10
e = 0.1
L = 3

In [9]:
##事前分布の設定
#階層モデルの事前分布
Deltabar = np.zeros((u_col, k))
ADelta =0.01 * np.diag(np.ones(u_col))
nu = k + 1
V = nu * np.diag(np.ones(k))

#モデルパラメータの事前分布
gamma1 = np.repeat(0, x_col)
tau1 = np.diag(np.repeat(100, x_col))
inv_tau1 = np.linalg.inv(tau1)
gamma2 = np.repeat(0, k*k)
tau2 = np.diag(np.repeat(100, k*k))
inv_tau2 = np.linalg.inv(tau2)

In [10]:
##データの設定
#切断領域を定義
a = (1-y)*(-100) + y*0
b = y*100 + (1-y)*0

#回帰モデルの定数を設定
xxv = np.dot(x.T, x) + inv_tau1
inv_xxv = np.linalg.inv(xxv)

#階層モデルの定数
uu = np.dot(u.T, u)
inv_uu = np.linalg.inv(np.dot(u.T, u) + ADelta)

#インデックスを設定
pt_m = pt*m
allocation_list1 = [i for i in range(d)]
for i in range(d):
    allocation_list1[i] = np.arange(pt[i]*m).reshape(pt[i], m, order="F")
allocation_list2 = np.arange(k*k).reshape(k, k, order="F")

In [11]:
##パラメータの真値
#階層モデルのパラメータの真値
alpha = alphat.copy()
alpha_mu = np.dot(u, alpha)
Cov = Covt.copy()
inv_Cov = np.linalg.inv(Cov)

#モデルパラメータの真値
Sigma = 1
beta = betat.copy()
theta = thetat.copy()
omega = omegat.copy()

#期待値の真値
beta_mu = np.dot(x, beta)
mu = np.zeros((hhpt, m))
for j in range(m):
    mu[:, j] = beta_mu[:, j] + np.dot(np.dot(theta, omega[:, :, j])[d_id1, ] * theta[d_id2, ], np.repeat(1, k))

In [12]:
##パラメータの初期値
#階層モデルのパラメータの初期値
alpha = np.random.normal(0, 0.2, u_col*k).reshape(u_col, k)
alpha_mu = np.dot(u, alpha)
Cov = np.diag(np.repeat(1.0, k))
inv_Cov = np.linalg.inv(Cov)

#モデルパラメータの初期値
Sigma = 1
beta = np.append(np.random.uniform(-1.0, -0.5, m), np.random.normal(0, 0.1, (x_col-1)*m)).reshape(x_col, m)
theta = np.random.multivariate_normal(np.repeat(0, k), np.diag(np.repeat(0.01, k)), d)
omega = np.random.normal(0, 0.1, k*k*m).reshape(k, k, m)

#期待値の初期値
beta_mu = np.dot(x, beta)
mu = np.zeros((hhpt, m))
for j in range(m):
    mu[:, j] = beta_mu[:, j] + np.dot(np.dot(theta, omega[:, :, j])[d_id1, ] * theta[d_id2, ], np.repeat(1, k))

In [13]:
##パラメータの格納用配列
#階層モデルの格納用配列
ALPHA = np.zeros((u_col, k, int(R/keep)))
COV = np.zeros((k, k, int(R/keep)))

#モデルパラメータの格納用配列
BETA = np.zeros((x_col, m, int(R/keep)))
THETA = np.zeros((d, k, int(R/keep)))
OMEGA = np.zeros((k, k, m, int(R/keep)))

In [14]:
##対数尤度の基準値
#1パラメータでの対数尤度
LLst1 = np.sum(y*np.full((hhpt, m), np.log(np.mean(y, axis=0))))
LLst0 = np.sum((1-y)*np.full((hhpt, m), np.log(np.mean(1-y, axis=0))))
LLst = LLst1 + LLst0
print(LLst)

#真値での対数尤度
mu = np.zeros((hhpt, m))
for j in range(m):
    mu[:, j] = np.dot(x, betat[:, j]) + np.dot(np.dot(thetat, omegat[:, :, j])[d_id1, ] * thetat[d_id2, ], np.repeat(1, k))
Prob = norm.cdf(mu, 0, Sigma)
Prob[Prob==1.0] = 0.99999999999; Prob[Prob==0] = np.power(10.0, -100)
LLbest = np.sum(y*np.log(Prob)) + np.sum((1-y)*np.log(1-Prob))
print(LLbest)

-2617950.092417704
-1672492.8452773914


In [ ]:
####ギブスサンプリングでパラメータをサンプリング####
for rp in range(R):
    ##切断正規分布から潜在変数をサンプリング
    #期待値の設定
    theta_vec1 = theta[d_id1, ]; theta_vec2 = theta[d_id2, ]
    beta_mu = np.dot(x, beta)
    ARA = np.zeros((hhpt, m))
    AR = [j for j in range(m)]; RA = [j for j in range(m)]
    for j in range(m):
        AR[j] = np.dot(theta, omega[:, :, j])[d_id1, ]; RA[j] = np.dot(theta, omega[:, :, j].T)[d_id2, ]
        ARA[:, j] = np.dot(AR[j] * theta_vec2, np.repeat(1, k))
    mu = beta_mu + ARA

    #切断領域に応じて潜在変数をサンプリング
    U = rtnorm(mu, Sigma, a, b, hhpt, m)


    ##素性ベクトルの回帰ベクトルをサンプリング
    #多変量正規分布のパラメータ
    er = U - ARA   #モデル誤差
    xy = np.dot(x.T, er)
    par_mu = np.dot(inv_xxv, xy)

    #多変量正規分布から回帰ベクトルをサンプリング
    for j in range(m):
        beta[:, j] = np.random.multivariate_normal(par_mu[:, j], inv_xxv, 1)
    beta_mu = np.dot(x, beta)
    

    ##ノードの特徴行列をサンプリング
    #モデル誤差を設定
    er = U - beta_mu; er_T = er.T
    for i in range(d):    
        #データの定義
        v = np.zeros((pt_m[i], k))
        for j in range(m):
            index = allocation_list1[i]
            v[index[:, j], ] = np.vstack((RA[j][d_list1[i], ], AR[j][d_list2[i], ]))

        #多変量正規分布のパラメータ
        vy = np.dot(v.T, er_T[:, d_list[i]].reshape(-1))
        vv = np.dot(v.T, v) + inv_Cov
        inv_vv = np.linalg.inv(vv)
        theta_mu = np.dot(inv_vv, vy + np.dot(inv_Cov, alpha_mu[i, ]))

        #多変量正規分布から特徴ベクトルをサンプリング
        theta[i, ] = np.random.multivariate_normal(theta_mu, inv_vv, 1).reshape(-1)


    ##omegaをサンプリング
    #双線形回帰行列の入力変数を設定
    theta_vec1 = theta[d_id1, ]; theta_vec2 = theta[d_id2, ]
    w = np.zeros((hhpt, k*k))
    for j in range(k):
        w[:, allocation_list2[:, j]] = theta_vec1[:, j][:, np.newaxis] * theta_vec2

    #多変量正規分布のパラメータ
    er = U - beta_mu   #モデル誤差
    wy = np.dot(w.T, er)
    wwv = np.dot(w.T, w) + inv_tau2
    inv_wwv = np.linalg.inv(wwv)
    omega_mu = np.dot(inv_wwv, wy)

    #多変量正規分布からomegaをサンプリング
    for j in range(m):
        omega[:, :, j] = np.random.multivariate_normal(omega_mu[:, j], inv_wwv, 1).reshape(k, k)


    ##階層モデルのパラメータをサンプリング
    #多変量回帰モデルから回帰行列と分散をサンプリング
    out = rmultireg(theta, u, inv_uu, np.dot(u.T, theta), Cov, ADelta, Deltabar, V, nu, d, u_col, k)    
    alpha = out[0]
    alpha_mu = np.dot(u, alpha)
    Cov = out[1]
    inv_Cov = np.linalg.inv(Cov)


    ##サンプリング結果の格納と表示
    if rp%keep==0:
        mkeep = int(rp/keep)
        ALPHA[:, :, mkeep] = alpha
        COV[:, :, mkeep] = Cov
        BETA[:, :, mkeep] = beta
        THETA[:, :, mkeep] = theta
        OMEGA[:, :, :, mkeep] = omega

    if rp%disp==0:
        #対数尤度を更新
        mu = np.zeros((hhpt, m))
        for j in range(m):
            mu[:, j] = np.dot(x, beta[:, j]) + np.dot(np.dot(theta, omega[:, :, j])[d_id1, ] * theta[d_id2, ], np.repeat(1, k))
        Prob = norm.cdf(mu, 0, Sigma)
        Prob[Prob==1.0] = 0.99999999999; Prob[Prob==0] = np.power(10.0, -100)
        LL = np.sum(y*np.log(Prob)) + np.sum((1-y)*np.log(1-Prob))
        
        #サンプリング結果の表示
        print(rp)
        print(np.round(np.array((LL, LLbest, LLst)), 1))

0
[-2598386.6 -1672492.8 -2617950.1]
10
[-1742223.8 -1672492.8 -2617950.1]
20
[-1690687.6 -1672492.8 -2617950.1]
30
[-1679430.3 -1672492.8 -2617950.1]
40
[-1675728.5 -1672492.8 -2617950.1]
50
[-1674016.4 -1672492.8 -2617950.1]
60
[-1673424.6 -1672492.8 -2617950.1]
70
[-1672724.7 -1672492.8 -2617950.1]
80
[-1672475.9 -1672492.8 -2617950.1]
90
[-1672453.9 -1672492.8 -2617950.1]
100
[-1672252.9 -1672492.8 -2617950.1]
110
[-1672376.3 -1672492.8 -2617950.1]
120
[-1672372.8 -1672492.8 -2617950.1]
130
[-1672375.2 -1672492.8 -2617950.1]
140
[-1672326.  -1672492.8 -2617950.1]
150
[-1671778.  -1672492.8 -2617950.1]
160
[-1672415.7 -1672492.8 -2617950.1]
170
[-1672535.4 -1672492.8 -2617950.1]
180
[-1672000.9 -1672492.8 -2617950.1]
190
[-1672197.4 -1672492.8 -2617950.1]
200
[-1671963.9 -1672492.8 -2617950.1]
210
[-1672147.4 -1672492.8 -2617950.1]
220
[-1672455.6 -1672492.8 -2617950.1]
230
[-1672156.  -1672492.8 -2617950.1]
240
[-1672246.4 -1672492.8 -2617950.1]
250
[-1672551.4 -1672492.8 -2617950.